In [1]:
## read text and preprocessing

import numpy as np
import re
import nltk

# download when needed
#nltk.download('stopwords')
#nltk.download('wordnet')

from sklearn.datasets import load_files
import pickle
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()
tfile = open('imdb_train.txt','r',encoding='utf-8')
Y = [] #label
x = [] #initial feature

#what the file looks like:
# 1 this movie is very good .......
for line in tfile:
    Y.append(line[0]) #take the 1
    x.append(line[2:])#take the text

tmp = [] #storing the processed x
for w in range(0, len(x)):
    # remove special characters
    txt = re.sub(r'\W', ' ', str(x[w]))
    
    # remove characters
    txt = re.sub(r'\s+[a-zA-Z]\s+', ' ', txt)
    
    # Remove single characters start
    txt = re.sub(r'\^[a-zA-Z]\s+', ' ', txt) 
    
    # remove multiple spaces with single space
    txt = re.sub(r'\s+', ' ', txt, flags=re.I)
    
    # removing prefixed 'b'
    txt = re.sub(r'^b\s+', '', txt)
    
    # turn lowercase
    txt = txt.lower()
    
    # lemmatization
    txt = txt.split()

    txt = [wnl.lemmatize(word) for word in txt]
    txt = ' '.join(txt)
    
    tmp.append(txt)
    
print(len(tmp))
print(tmp[0])
print(Y[1:5])

25000
this film wa just brilliant casting location scenery story direction everyone really suited the part they played and you could just imagine being there robert redford is an amazing actor and now the same being director norman father came from the same scottish island a myself so loved the fact there wa real connection with this film the witty remark throughout the film were great it wa just brilliant so much that bought the film a soon a it wa released for retail and would recommend it to everyone to watch and the fly fishing wa amazing really cried at the end it wa so sad and you know what they say if you cry at film it must have been good and this definitely wa also congratulation to the two little boy that played the part of norman and paul they were just brilliant child are often left out of the praising list think because the star that play them all grown up are such big profile for the whole film but these child are amazing and should be praised for what they have done don 

## turn text into num

In [2]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

# initialize vectorizer
# max_features = bag of words method, 1500 as most occuring words
# mind_df = only include features that appear in at least 5 docs
# max_df = like min_df, 0.7 means 70%
# remove stopwords in file from nltk lib
vtrizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
# turn words to nums
# fit_transform coverts text into num features
X = vtrizer.fit_transform(tmp).toarray()

# TFIDF only useful when your text files are separated
# TFIDF assign score across documents
#from sklearn.feature_extraction.text import TfidfTransformer
#X = TfidfTransformer.fit_transform(X1).toarray()

print(len(X))
print(X[2])

25000
[0 0 0 ... 0 0 0]


# split dataset to training and test set

In [3]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y, test_size=0.2, random_state=0)

# model training and testing

In [4]:
from sklearn.ensemble import RandomForestClassifier
bc = RandomForestClassifier(n_estimators=1000, random_state=0)
bc.fit(xtrain, ytrain)
ypred = bc.predict(xtest)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(ytest,ypred))
print(classification_report(ytest,ypred))
print(accuracy_score(ytest, ypred))

[[2065  412]
 [ 371 2152]]
              precision    recall  f1-score   support

           0       0.85      0.83      0.84      2477
           1       0.84      0.85      0.85      2523

   micro avg       0.84      0.84      0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000

0.8434


# andreas revisited

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def create_model():
  model = Sequential()
  model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

c:\users\kek\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
c:\users\kek\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
c:\users\kek\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Error when checking input: expected dense_4_input to have shape (60,) but got array with shape (1500,)